# Lab 3: Logistics

Closed circuit footage from one of the vandalism sites:

![zebra approaching](./img/cctv.png)

This has caused a bit of a stir. Overnight, all of the zoos in the world report that their zebras have gone missing without a trace. The previous benign view of zebras has been replaced with a harsh potential reality:

![harsh reality](./img/achtung.png)

While we don't yet know what has provoked zebras into acts of criminal vandalism, we nevertheless must take steps to defend our innocent global walls. The United Nations Commission Opposed to Obscure Lawlessness has been asked to protect the world's various important walls by automatically detecting zebras from camera footage.

Your team has compiled a zipped database of zebra photography that you should download from the following link: 

https://drive.google.com/file/d/1l5DgKp3zKZ1O9bpESwVV7Y2O7DEQLMg5/view?usp=sharing

After downloading the .zip file, you should move it to the directory containing this Python notebook and then unzip it.

Your team has also provided helpful utility functions in `uncool.py`. Some of these functions read and display images.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from uncool import ZEBRA_TRAIN_FILES, load_image, show_image
img = load_image(ZEBRA_TRAIN_FILES[0])
show_image(img)

To avoid false alarms, you want to be able to distinguish zebra terrorists from peace-loving horses. Therefore your team has also compiled a database of horse photography.

In [ ]:
from uncool import HORSE_TRAIN_FILES, load_image, show_image
img = load_image(HORSE_TRAIN_FILES[16])
show_image(img)

Your plan is to train a logistic regression model that can distinguish zebras from horses. Then, you can hook up security cameras to alarms that will go off if zebras (but not horses) are detected near important cultural sites.

Since this is a binary classification task (zebra or no zebra), you decide to use logistic regression. 

----
## Question 1
----

In order to use logistic regression, you first need a way to convert images into feature vectors. You brainstorm with your UNCOOL colleagues and come up with a few possible features for distinguishing zebras from horses:

- the percentage of black pixels in the image
- the percentage of white pixels in the image
- the percentage of nearby pixels that have a high contrast (i.e. one is white and one is black)

You can test your feature implementations on a grayscale version of the Swiss flag:

In [ ]:
from test import SWISS_FLAG
show_image(SWISS_FLAG)

You will need to complete two functions (both in ```logistic.py```):
- ```count_pixel_percent```: This takes two arguments: a grayscale image ```img``` (an order-2 torch tensor) and ```predicate``` (a function that maps a pixel intensity -- an integer between 0 and 255 -- to a Boolean value). ```count_pixel_percent``` should return the percentage of image pixels for which ```predicate(pixel)``` returns ```True```.
- ```count_contrasting_pixels```: This takes three arguments: a grayscale image ```img``` (an order-2 torch tensor), an integer ```stride```, and an integer ```threshold```. This function examines every pair of image pixels ```(x,y)``` and ```(x+stride,y)```. It returns the percentage of these pairs for are "high contrast" -- i.e. the absolute difference between the intensities of the two pixels is greater than or equal to ```threshold```.

We've provided a unit test in test.py, so that you can check your 
implementation is working properly. Run it from the command line as follows:

```python -m unittest test.Q1```

Once you can convert images into features, it's time to convert your zebra and horse photos into a training dataset. Your team has created code for this in ```uncool.py```. It uses the ```pandas``` package to create a ```DataFrame``` that stores the features and responses for each image, e.g. if we run the code on the first 3 zebra and horse images in the training set, we should get:

```
   bias     black  contrast     white  zebra
0   1.0  0.151310  0.054470  0.094945      1
1   1.0  0.158325  0.011257  0.193766      1
2   1.0  0.023300  0.010792  0.453148      1
3   1.0  0.165342  0.009893  0.062728      0
4   1.0  0.236591  0.010355  0.018503      0
5   1.0  0.253384  0.017671  0.155455      0
```

You can replicate this by running:

In [ ]:
from logistic import count_pixel_percent, count_contrasting_pixels
from uncool import create_dataframe, extract_image_features
features = {'bias': lambda img: 1.0,
            'black': lambda img: count_pixel_percent(img, lambda pixel: pixel < 50),
            'contrast': lambda img: count_contrasting_pixels(img, 2, 130),            
            'white': lambda img: count_pixel_percent(img, lambda pixel: pixel > 160)}
dataframe = create_dataframe(lambda im: extract_image_features(im, features),
                             ZEBRA_TRAIN_FILES[:3], 
                             HORSE_TRAIN_FILES[:3])
print(dataframe)

Now go ahead and convert all your training image data into a dataframe, and then save it to disk as a comma-separated values (CSV) file. 

In [ ]:
from uncool import compile_dataset
compile_dataset(features,
                ZEBRA_TRAIN_FILES, 
                HORSE_TRAIN_FILES,
                "simple.train.csv")

This may take a while if your feature implementations are inefficient (by contrast, if your implementations are efficient, the above operation should take less than a second). This is a good opportunity to refine your ```torch``` skills. The functions ```count_pixel_percent``` and ```count_contrasting_pixels``` can be implemented much more efficiently by the expeditious use of tensor operations (and by **completely** avoiding Python loops).

Your UNCOOL team suggests that the following [torch operations](https://pytorch.org/docs/stable/torch.html) could be useful:
- ```torch.abs``` ([documentation](https://pytorch.org/docs/stable/generated/torch.abs.html))
- ```torch.Tensor.item``` ([documentation](https://pytorch.org/docs/stable/generated/torch.Tensor.item.html))
- ```torch.numel``` ([documentation](https://pytorch.org/docs/stable/generated/torch.numel.html))
- ```torch.sub``` ([documentation](https://pytorch.org/docs/stable/generated/torch.sub.html))
- ```torch.sum``` ([documentation](https://pytorch.org/docs/stable/generated/torch.sum.html))

Additionally, don't forget that tensors can be sliced in a similar manner as Python lists. Moreover, Boolean predicates can be applied rapidly to each element of a ```torch``` tensor:


In [ ]:
from torch import tensor
t = tensor([[5, 7, 9, 11],
            [6, 8, 10, 12],
            [7, 9, 11, 13]])
print(t > 8)

In [ ]:
def is_even(num):
    return num % 2 == 0

print(is_even(t))

Once you are happy with the efficiency of your functions, you also compile a separate test set of zebra and horse photos, and build a dataset out of those.

In [ ]:
from uncool import compile_dataset, ZEBRA_TEST_FILES, HORSE_TEST_FILES
compile_dataset(features, 
                ZEBRA_TEST_FILES, 
                HORSE_TEST_FILES,
                "simple.test.csv")

----
## Question 2
----

The next thing you need to do is to create code for a logistic regression model. At first, you will assume that you've already trained a parameter vector and all you want to do is run the logistic regression model on some new feature vectors to predict the responses. For this purpose, you create a class called ```LogisticRegressionModel```.

Complete the implementation of ```LogisticRegressionModel``` according to the comments. Specifically, you need to implement three methods: ```predict_probs```, ```classify```, and ```evaluate```.

```predict_probs``` takes a feature matrix as its argument. It maps each feature vector of the feature matrix to the probability that the corresponding response is equal to 1. Thus the return value is a vector of probabilities.

```classify``` takes a feature matrix as its argument. It maps each feature vector of the matrix to 1 if the probability of a positive corresponding response (according to ```predict_probs```) is greater than or equal to some threshold (which is 0.5 by default). Otherwise it maps the feature vector to 0. Thus the return value is a vector of zeroes and ones.

```evaluate``` takes a feature matrix ```X``` and a response vector ```y``` as arguments. It computes ```classify(X)``` and compares the resulting vector with ```y```, returning the percentage of elements that are (elementwise) equivalent.

In [ ]:
import torch
from logistic import LogisticRegressionModel
params = torch.tensor([[0., 0., 0.], 
                       [2., 3., 2.5]])
model = LogisticRegressionModel(params)
X = torch.tensor([[ 1.0000,  1.0000,  1.0000,  1.0000],
                  [ 0.5000, -1.4000,  1.2000, -1.8000],
                  [-1.2000, -0.4000,  2.0000,  2.2000]])
print(model.predict_probs(X))  # should return tensor([0.6225, 0.0392, 1.0000, 0.8909])
print(model.classify(X))       # should return tensor([1, 0, 1, 1], dtype=torch.int32)
print(model.evaluate(X, torch.tensor([0, 1, 0, 1])))  # should return 0.25
print(model.evaluate(X, torch.tensor([1, 0, 0, 1])))  # should return 0.75

We've provided a unit test in ```test.py```, so that you can check your 
implementation is working properly. Run it from the command line as follows:

```python -m unittest test.Q2```

----
## Question 3
----

Now it is time to implement the training of the logistic regression model. As with lasso regression, there isn't a "closed form" solution to the optimization problem, so we have to use gradient descent. Luckily, we have existing implementations of gradient descent from our conquest of Chekhov's Sun, which we've put in ```descent.py```. Again, you figure that you'll use adagrad.

Complete the function ```train_logistic_regression``` in ```logistic.py```. It should take two arguments: a feature matrix ```X``` and a response vector ```y```. It should return a trained ```LogisticRegressionModel```.

The basic implementation is already provided, but (similar to ```LassoSpace``` from the previous lab) you need to implement a class called ```LogisticRegressionParameterSpace``` that implements the ```ParameterSpace``` abstract class for logistic regression.

**All you really have to do is complete the ```gradient``` method.**

We've provided a unit test in ```test.py```, so that you can check your function is working properly. Run it from the command line as follows:

    python -m unittest test.Q3
    
Once it's working, we can train it on our training data and evaluate its accuracy on our test data.

In [ ]:
from uncool import load_data
from logistic import train_logistic_regression
X_train, y_train = load_data('simple.train.csv')
lr = train_logistic_regression(X_train, y_train)
X_test, y_test = load_data('simple.test.csv') 
print('Accuracy of trained model on test data: {:.3f}'.format(
    lr.evaluate(X_test, y_test)))

That's not bad! Even with a simple model, we're making mistakes less than 30% of the time. Still, that still leaves a lot of unhindered vandalism (not to mention several scandalized horses). Perhaps you can do better if you brainstorm more features with your UNCOOL friends.

----
## Question 4
----

After trying your logistic regression with subsets of your original features (something that data scientists like yourself call an "ablation study"), you realize that the most important feature is ```contrast```, which indicates how many pixel pairs ```(x,y)``` and ```(x+2,y)``` have high-contrast coloration.

But this only works for zebras of a particular orientation. What if a zebra is tilted a bit? What if the zebra is lying down? Then the stripes would be horizontal and we'd want to look at pixel pairs ```(x,y)``` and ```(x,y+2)```.

Rather than worry about it, you implement a general-purpose function that can count what your committee decides to call "pixel patterns".

It first collects a histogram over pixel patterns. For instance, if the pixel pattern is ```[(0,0), (0,2), (2,1)]```, then the feature extractor looks at all image pixel triples ```(x,y)``` and ```(x,y+2)``` and ```(x+2, y+1)```. It would count the patterns ```p_ijk```, where ```i = 1``` if pixel ```(x,y)``` exceeds a color threshold (otherwise ```i=0```) and ```j = 1``` if pixel ```(x, y+2)``` exceeds the color threshold (otherwise ```j=0```) and ```k = 1``` if pixel ```(x+2, y+1)``` exceeds the color threshold (otherwise ```k=0```).

For instance, for the ```SWISS_FLAG``` image, the histogram (given a threshold of 130) would be:

    {'p_000': 0, 'p_001': 4, 'p_010': 2, 'p_011': 0,
     'p_100': 2, 'p_101': 0, 'p_110': 1, 'p_111': 0}

since (for instance) there are 4 pixel triples such that ```(x,y)``` and ```(x,y+2)``` are black while ```(x+2, y+1)``` is white. 

**Your task:** complete the class ```extract_pixel_pattern_features``` in ```logistic.py``` according to its comments and the above description of how it should work.

We've provided a unit test in ```test.py```, so that you can check your function is working properly. Run it from the command line as follows:

    python -m unittest test.Q4
   
Be aware that efficiency is very important for this function. If the function is implemented with nested Python loops, it may take hours to compile the training data into features.

Your UNCOOL team suggests that tensor slicing and the following [torch operations](https://pytorch.org/docs/stable/torch.html) could be useful:
- ```torch.Tensor.item``` ([documentation](https://pytorch.org/docs/stable/generated/torch.Tensor.item.html))
- ```torch.reshape``` ([documentation](https://pytorch.org/docs/stable/generated/torch.reshape.html))
- ```torch.stack``` ([documentation](https://pytorch.org/docs/stable/generated/torch.stack.html))
- ```torch.transpose``` ([documentation](https://pytorch.org/docs/stable/generated/torch.transpose.html))
- ```torch.unique``` ([documentation](https://pytorch.org/docs/stable/generated/torch.unique.html)), particularly with the flag ```return_counts=True``` 

Also, `itertools.product` can be useful for generating all possible bitstrings of a certain length.

**Warning:** an efficient implementation may be challenging.

Once this is working, you're ready to try out these new features on zebra detection. First you create the data (this may take a minute or two, even with a relatively efficient implementation):

In [ ]:
from logistic import extract_pixel_pattern_features

def feature_extractor(im):
    pattern = [(0,0), (1,0), (0,1), (2,0), (0,2), 
               (1,1), (2,2), (0,20), (20, 0)]
    histogram = extract_pixel_pattern_features(im, 130, pattern)
    pattern_count = sum([histogram[p] for p in histogram])
    normalized_pattern_features = {p: histogram[p] / pattern_count 
                                   for p in histogram}
    return {**normalized_pattern_features, 'bias': 1.0}
    
train_df = create_dataframe(feature_extractor, 
                            ZEBRA_TRAIN_FILES, 
                            HORSE_TRAIN_FILES)
train_df.to_csv('pattern.train.csv') 

test_df = create_dataframe(feature_extractor, 
                           ZEBRA_TEST_FILES, 
                           HORSE_TEST_FILES)
test_df.to_csv('pattern.test.csv')      


Then you train it on the training data and evaluate its accuracy on the test data.

In [ ]:
from uncool import load_data
from logistic import train_logistic_regression
X_train, y_train = load_data('pattern.train.csv') 
lr = train_logistic_regression(X_train, y_train)
X_test, y_test = load_data('pattern.test.csv') 
print('Accuracy of trained model on test data: {:.3f}'.format(
    lr.evaluate(X_test, y_test)))

Great! Your accuracy has (hopefully) jumped from less than 72% to over 81%. 

----
## Question 5
----


But what kind of mistakes are you making? There are two types of errors you could be making. A **false positive** is when the classifier says there's a zebra, but really there isn't. A **false negative** is when the classifier says there's no zebra, but really there is. These two errors have different consequences. A **false positive** is a little annoying, because it triggers a false alarm and frightens horses. But a **false negative** allows zebras to slip by our security undetected.

Time to investigate this question a bit deeper. **Precision** is the technical term for the fraction of the time that the classifier is correct, when it makes a positive prediction. In other words, if the classifier has high precision (is highly precise), then it almost never predicts false positives. **Recall** is the term for the fraction of the time that the classifier is correct, when it is classifying a positive instance. In other words, if the classifier has high recall, then it rarely predicts false negatives.

Complete the functions ```precision``` and ```recall``` in 
```logistic.py``` according to their comments.

Your UNCOOL team suggests that the following [torch operations](https://pytorch.org/docs/stable/torch.html) could be useful:
- ```torch.flatten``` ([documentation](https://pytorch.org/docs/stable/generated/torch.flatten.html))
- ```torch.index_select``` ([documentation](https://pytorch.org/docs/stable/generated/torch.index_select.html))
- ```torch.Tensor.item``` ([documentation](https://pytorch.org/docs/stable/generated/torch.Tensor.item.html))
- ```torch.nonzero``` ([documentation](https://pytorch.org/docs/stable/generated/torch.nonzero.html))
- ```torch.sum``` ([documentation](https://pytorch.org/docs/stable/generated/torch.sum.html))

We've provided a unit test in ```test.py```, so that you can check your functions are working properly. Run it from the command line as follows:

    python -m unittest test.Q5
   
Once this is working, you can compute the precision and recall of your trained logistic regression classifier on the test set.

In [ ]:
from logistic import precision, recall
print("Precision: {}".format(precision(lr, X_test, y_test)))
print("Recall: {}".format(recall(lr, X_test, y_test)))

Interesting. If you hook up your zebra detector to an alarm system, then it'll sound a false alarm for (only) a bit more than 17% of the horses that pass by. Unfortunately, it will also completely miss more than 20% of the zebras.

You've been told that the international powers are ok with scaring horses, as long as more zebras are detained. Luckily, by calibrating the threshold of the logistic regression classifier, you can make this tradeoff. Right now, you're classifying an image as a zebra if the model's probability is greater than 0.5. But alternatively, you could classify an image as a zebra if the model's probability is greater than 0.2. This would cause more false positives (frightened horses) but would reduce the number of true negatives (undetected zebras).

One way to visualize the tradeoff is to plot the precision versus the recall for various thresholds. This is called a **PR (Precision-Recall) Curve**.

In [ ]:
from logistic import precision, recall

def pr_curve(X, y):
    precisions = []
    recalls = []
    for thres in [0.001 * i for i in range(1,1000)]:
        prec = precision(lr, X, y, thres)
        rec = recall(lr, X, y, thres)
        precisions.append(prec)
        recalls.append(rec)
    return precisions, recalls

import matplotlib.pyplot as plt
%matplotlib inline
p, r = pr_curve(X_test, y_test)
plt.plot(r, p)
plt.xlabel('recall')
plt.ylabel('precision')
plt.title('Precision-Recall Curve')

The curve indicates that we can detect (recall) almost 90% of zebras, as long as we're willing to trigger an alarm for something like 30% of passing horses (i.e. tolerate a precision of roughly 70%). You consult the powers that be.

They're willing to make that sacrifice.

**Lab 3 is now complete. Push your code to receive credit. Do not add the image files to your commit.**